In [1]:
import pandas as pd
from ete3 import NCBITaxa

## get picked taxid with its genetic code

In [2]:
filepath="~/altorf/genome/speciespick/picked_assembly_summary_code.csv"
df=pd.read_csv(filepath)
print(df.shape)
df.head()

(1615, 8)


,asm_name,assembly_accession,domain,ftp_basename,ftp_path,organism_name,taxid,genetic_code
0,ASM2238v1,GCF_000022385.1,archaea,GCF_000022385.1_ASM2238v1,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,Sulfolobus islandicus L.S.2.15,429572,11
1,ASM97008v1,GCF_000970085.1,archaea,GCF_000970085.1_ASM97008v1,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,Methanosarcina siciliae T4/M,1434120,11
2,ASM734v1,GCF_000007345.1,archaea,GCF_000007345.1_ASM734v1,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,Methanosarcina acetivorans C2A,188937,11
3,ASM35030v1,GCF_000350305.1,archaea,GCF_000350305.1_ASM35030v1,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,Thermoplasmatales archaeon BRNA1,1054217,11
4,ASM96990v1,GCF_000969905.1,archaea,GCF_000969905.1_ASM96990v1,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,Methanosarcina vacuolata Z-761,1434123,11


## get corresponding parent taxids

In [3]:
def get_desired_ranks(taxid, desired_ranks):
    lineage = ncbi.get_lineage(taxid)
    lineage2ranks = ncbi.get_rank(lineage)
    ranks2lineage = dict((rank, taxid) for (taxid, rank) in lineage2ranks.items())
    return {str(rank): ranks2lineage.get(rank, '-1') for rank in desired_ranks}

ncbi = NCBITaxa()
desired_ranks = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']

dct_lst=[]
for taxid in df["taxid"]:
    dct=get_desired_ranks(taxid, desired_ranks)
    dct["taxid"]=taxid
    dct_lst.append(dct)
tax_df=pd.DataFrame(dct_lst)
tax_df=tax_df[["taxid"]+desired_ranks]
print(tax_df.shape)
tax_df.head()

(1615, 8)


,taxid,kingdom,phylum,class,order,family,genus,species
0,429572,-1,28889,183924,2281,118883,2284,43080
1,1434120,-1,28890,224756,94695,2206,2207,38027
2,188937,-1,28890,224756,94695,2206,2207,2214
3,1054217,-1,28890,183967,2301,-1,-1,1054217
4,1434123,-1,28890,224756,94695,2206,2207,2215


In [5]:
for rank in (["taxid"]+desired_ranks):
    print("{0}: {1}".format(rank,len(set(tax_df[rank]))))

taxid: 1615
kingdom: 1
phylum: 37
class: 69
order: 155
family: 314
genus: 794
species: 1577


## get AveNSRF information

In [6]:
countFilepath="~/altorf/genome/patternanalyze/summarize_count.csv"
count_df=pd.read_csv(countFilepath)
print(count_df.shape)
count_df.head()

(1615, 3)


,taxid,count_real,count_sim
0,429572,0.425893,0.427635
1,1434120,0.463257,0.442056
2,188937,0.448898,0.429019
3,1054217,1.094285,0.843878
4,1434123,0.410412,0.401882


## get compositional information

In [7]:
compFilepath="~/altorf/genome/preprocess/out/summary_composition.csv"
comp_df=pd.read_csv(compFilepath)
comp_df=comp_df[comp_df["dna_type"]=="chromosome"]#exclude plasmid
print(comp_df.shape)
comp_df.head()

(1615, 100)


,ftp_basename,dna_type,G+C,num_ambiguous,per_ambiguous,A,C,G,T,B,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,GCF_000022385.1_ASM2238v1,chromosome,0.351110,0,0.0,888038,475131,485602,887501,NaN,...,14898,59802,45239,25365,35234,33745,95524,58476,47604,96167
1,GCF_000970085.1_ASM97008v1,chromosome,0.429404,0,0.0,1420708,1073050,1081510,1442290,NaN,...,52223,96142,93249,75156,63340,72350,91433,129173,89002,192140
2,GCF_000007345.1_ASM734v1,chromosome,0.426809,0,0.0,1638004,1228410,1226378,1658700,NaN,...,60400,112816,108481,86152,72519,82104,104484,149528,102469,220277
3,GCF_000350305.1_ASM35030v1,chromosome,0.583030,0,0.0,306724,427580,424288,302513,NaN,...,37083,22944,22468,21107,24589,17132,5507,26971,14986,9466
4,GCF_000969905.1_ASM96990v1,chromosome,0.397795,0,0.0,1340341,902862,889505,1373044,NaN,...,39862,94306,85991,64803,56747,65669,99640,117838,85862,191079


In [8]:
out_df=pd.merge(tax_df, count_df, on="taxid",how="left")
out_df["diff"]=out_df["count_real"]-out_df["count_sim"]

out_df=pd.merge(out_df, df[["taxid","ftp_basename","organism_name","genetic_code"]], on="taxid",how="left")#!TBI! because the key of comp_df is not taxid bt ftp_basename...
out_df=pd.merge(out_df, comp_df[["ftp_basename","G+C"]],on="ftp_basename",how="left")
print(out_df.shape)
out_df.head()

(1615, 15)


,taxid,kingdom,phylum,class,order,family,genus,species,count_real,count_sim,diff,ftp_basename,organism_name,genetic_code,G+C
0,429572,-1,28889,183924,2281,118883,2284,43080,0.425893,0.427635,-0.001742,GCF_000022385.1_ASM2238v1,Sulfolobus islandicus L.S.2.15,11,0.351110
1,1434120,-1,28890,224756,94695,2206,2207,38027,0.463257,0.442056,0.021201,GCF_000970085.1_ASM97008v1,Methanosarcina siciliae T4/M,11,0.429404
2,188937,-1,28890,224756,94695,2206,2207,2214,0.448898,0.429019,0.019879,GCF_000007345.1_ASM734v1,Methanosarcina acetivorans C2A,11,0.426809
3,1054217,-1,28890,183967,2301,-1,-1,1054217,1.094285,0.843878,0.250406,GCF_000350305.1_ASM35030v1,Thermoplasmatales archaeon BRNA1,11,0.583030
4,1434123,-1,28890,224756,94695,2206,2207,2215,0.410412,0.401882,0.008531,GCF_000969905.1_ASM96990v1,Methanosarcina vacuolata Z-761,11,0.397795


In [9]:
genus_lst=list(set(tax_df["genus"]))
dct_lst=[]
for genus in genus_lst:
    dct={}
    dct["genus"]=genus
    filtered_df=out_df[out_df["genus"]==genus]
    dct["#"]=len(filtered_df)
    dct["mean_diff"]=filtered_df["diff"].mean()
    dct["mean_G+C"]=filtered_df["G+C"].mean()
    if len(filtered_df)==(filtered_df["genetic_code"]==11).sum():
        dct["genetic_code"]=11
    else:
        dct["genetic_code"]=-1
    dct_lst.append(dct)
genus_df=pd.DataFrame(dct_lst)
genus_df.head()

,#,genetic_code,genus,mean_G+C,mean_diff
0,1,11,2050,0.554133,0.044594
1,3,11,2053,0.673006,0.142491
2,1,11,6,0.673190,0.099512
3,1,11,2056,0.681094,0.033561
4,1,11,10,0.519912,0.043912


In [10]:
genus_df=genus_df.sort_values(by=["#","mean_diff"],ascending=False)
genus_df.head(20)

,#,genetic_code,genus,mean_G+C,mean_diff
687,35,11,1716,0.609289,0.072668
20,30,-1,2093,0.287322,0.000020
652,29,11,-1,0.481228,0.079529
641,27,11,1578,0.410088,-0.000174
753,21,11,1883,0.722731,0.170975
568,21,11,1386,0.409670,0.003619
607,20,11,1485,0.309281,0.004572
147,18,11,286,0.629517,0.141646
706,18,11,1763,0.670261,0.103016
543,15,11,1301,0.390050,-0.000120


genus_id=872, 286, 1678 seems good for test case analysis

In [11]:
(out_df[out_df["genus"]==872]).to_csv("../out/target_genus.list",index=False)

In [12]:
out_df[out_df["genus"]==872]

,taxid,kingdom,phylum,class,order,family,genus,species,count_real,count_sim,diff,ftp_basename,organism_name,genetic_code,G+C
304,525146,-1,1224,28221,213115,194924,872,876,0.851789,0.777740,0.074048,GCF_000022125.1_ASM2212v1,Desulfovibrio desulfuricans subsp. desulfurica...,11,0.580722
316,883,-1,1224,28221,213115,194924,872,881,1.619893,1.407806,0.212088,GCF_000021385.1_ASM2138v1,Desulfovibrio vulgaris str. 'Miyazaki F',11,0.671090
320,901,-1,1224,28221,213115,194924,872,901,1.387866,1.183206,0.204661,GCF_900116045.1_DESPIGER,Desulfovibrio piger,11,0.641799
503,526222,-1,1224,28221,213115,194924,872,880,0.594646,0.536652,0.057994,GCF_000023445.1_ASM2344v1,Desulfovibrio salexigens DSM 2638,11,0.470928
604,641491,-1,1224,28221,213115,194924,872,876,1.622397,1.369687,0.252709,GCF_000189295.2_ASM18929v2,Desulfovibrio desulfuricans ND132,11,0.652094
644,690850,-1,1224,28221,213115,194924,872,873,0.950594,0.790199,0.160394,GCF_000212675.2_ASM21267v2,Desulfovibrio africanus str. Walvis Bay,11,0.614208
652,207559,-1,1224,28221,213115,194924,872,58180,0.886230,0.786240,0.099990,GCF_000012665.1_ASM1266v1,Desulfovibrio alaskensis G20,11,0.578361
847,44742,-1,1224,28221,213115,194924,872,44742,1.071474,0.939710,0.131764,GCF_001553605.1_ASM155360v1,Desulfovibrio fairfieldensis,11,0.609001
986,643562,-1,1224,28221,213115,194924,872,182210,1.199880,1.026983,0.172897,GCF_000176915.2_ASM17691v2,Desulfovibrio aespoeensis Aspo-2,11,0.625620
1604,573370,-1,1224,28221,213115,194924,872,184917,1.227576,0.969581,0.257995,GCF_000010665.1_ASM1066v1,Desulfovibrio magneticus RS-1,11,0.627694
